# Building Chatbot that will answer in behalf of Sora(no game no life)!

In [61]:

from dotenv import load_dotenv
from pypdf import PdfReader
import requests
import json
from IPython.display import Markdown, display
import gradio as gr
from pathlib import Path
import os


load_dotenv(override=True)

api_key=os.getenv("API_KEY")

if api_key:
    print(f"api key exists = {api_key[:14]}")
else:
    print("api key not set")
    

#Settings
openrouter_base_url="https://openrouter.ai/api/v1/chat/completions"
model="openai/gpt-oss-20b:free"

api key exists = sk-or-v1-eaf09


In [62]:
# Read the Profile PDF
pdfReader = PdfReader("..\\resources\\Profile.pdf")
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

In [63]:
import os

# Get the directory of the current script
script_dir = Path.cwd().parent

# Build the relative path from the script's directory
summ_filePath = os.path.join(script_dir, "resources", "Summary.txt")
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

print(f"File path used: {summ_filePath}")  # Very helpful for debugging!


File path used: c:\Users\marvi\workspace\agentic_ai\resources\Summary.txt


In [ ]:
# System Prompt
system_message = f"""
You are Sora from no game no life.
Roleplay as Sora from No Game No Life

all interactions should Be in the role of Sora
Heres a summary:
\n{summary}\n
heres a profile summary:
\n{prof_summary}\n
"""

In [65]:
history = []
prompt = ""

def liveChat(prompt, history):
    # Build the messages list
    messages = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": prompt}
    ]

    # Send request to OpenRouter
    response = requests.post(
        url=openrouter_base_url,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": model,
            "messages": messages
        })
    )

    # Parse JSON
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

    try:
        data = response.json()
        raw_response = data["choices"][0]["message"]["content"]

        # Update history with user and assistant messages
        #history.append({"role": "user", "content": prompt})
        #history.append({"role": "assistant", "content": raw_response})

        return raw_response
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        return None


In [66]:
gr.ChatInterface(liveChat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
